In [1]:
!pip install haversine

In [2]:
import pandas as pd
import numpy as np
from haversine import haversine, Unit
import openpyxl

In [3]:
subway_df = pd.read_csv('../data/전처리데이터/성동구 지하철.csv',encoding='cp949')
bus_df= pd.read_csv('../data/전처리데이터/성동구 쉼터여부_버스 승차수_버스수_배차간격.csv',encoding='cp949')
bus_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
bus_df

,ARS-ID,버스 총 승차수,쉼터여부,NODE_ID,정류소명,위도,경도,버스 수,버스 노선 리스트,평균 배차 간격,최대 배차 간격
0,4001,39270,0,103000001,답십리역사거리,37.565426,127.054368,6,"['9403', '370', '303', 'N30', '721', '130']",13.67,40
1,4002,37286,0,103000002,청년회의소.서울새활용플라자,37.561952,127.060479,9,"['9403', '3216', '2221', '370', '3220', '303',...",13.89,40
2,4003,61371,0,103000003,장한평역,37.561131,127.066098,10,"['9403', '2416', '3216', '2221', '370', '3220'...",14.00,40
3,4101,17969,1,103000004,상왕십리역,37.564279,127.029260,5,"['2012', 'N62', '2014', '2015', '463']",12.80,24
4,4102,35007,1,103000005,상왕십리역,37.564609,127.028642,11,"['302', '2013', '2012', 'N62', '2014', '463', ...",12.18,24
...,...,...,...,...,...,...,...,...,...,...,...
427,4911,454,0,103900293,한양대학교정문,37.557954,127.041932,1,['성동03-2'],14.00,14
428,4912,754,0,103900284,한양대학교제2의학관,37.558815,127.041924,1,['성동03-2'],14.00,14
429,4913,264,0,103900292,한양대학교동문회관,37.558833,127.041823,1,['성동03-2'],14.00,14
430,4914,2972,0,103900283,한양대학교정문,37.557771,127.042058,1,['성동03-2'],14.00,14


In [4]:
distance_compare_min = []
min_distance_staion = []

## for문으로 비교
for i in range(len(bus_df)):
    bus_stop = (bus_df['위도'][i], bus_df['경도'][i])
    distance = 999999
    near_station = ''
    for j in range(len(subway_df)):
        station = (subway_df['sub_lat'][j], subway_df['sub_lng'][j])
        tmp_distance = np.round(float(haversine(bus_stop, station, unit='m')), 3)
        if tmp_distance < distance:
            distance = tmp_distance
            near_station = subway_df['역명'][j]
    distance_compare_min.append(distance)
    min_distance_staion.append(near_station)
    # print(bus_df['정류소명'][i], near_station, distance)

In [5]:
bus_df['가장 가까운 지하철역'] = min_distance_staion
bus_df['지하철역 최단거리(m)'] = distance_compare_min

In [6]:
bus_df.head()

,ARS-ID,버스 총 승차수,쉼터여부,NODE_ID,정류소명,위도,경도,버스 수,버스 노선 리스트,평균 배차 간격,최대 배차 간격,가장 가까운 지하철역,지하철역 최단거리(m)
0,4001,39270,0,103000001,답십리역사거리,37.565426,127.054368,6,"['9403', '370', '303', 'N30', '721', '130']",13.67,40,답십리,217.156
1,4002,37286,0,103000002,청년회의소.서울새활용플라자,37.561952,127.060479,9,"['9403', '3216', '2221', '370', '3220', '303',...",13.89,40,장한평,367.819
2,4003,61371,0,103000003,장한평역,37.561131,127.066098,10,"['9403', '2416', '3216', '2221', '370', '3220'...",14.00,40,장한평,136.266
3,4101,17969,1,103000004,상왕십리역,37.564279,127.029260,5,"['2012', 'N62', '2014', '2015', '463']",12.80,24,상왕십리,42.408
4,4102,35007,1,103000005,상왕십리역,37.564609,127.028642,11,"['302', '2013', '2012', 'N62', '2014', '463', ...",12.18,24,상왕십리,23.403


In [7]:
bus_df.to_csv("../data/전처리데이터/성동구 쉼터여부_버스 승차수_버스수_배차간격_지하철역최단거리.csv", encoding='cp949')